In [ ]:
%%capture
from fastai_transformers_utils.general import TransformersTokenizer

from typing import *
from fastai2.basics import *
from fastai2.text.all import *

from transformers import AutoTokenizer

In [ ]:
# default_exp tmp

# module name here

> API details.

In [ ]:
encoder_model_name = 'hfl/chinese-bert-wwm-ext'
decoder_model_name = 'distilgpt2'
seed = 42
ori_data_loc = '../data/cmn.txt'
tok_data_loc = '../data/tok_cmn.csv'

## Look at Data

In [ ]:
df = pd.read_csv(ori_data_loc, header=None, names=['English', 'Chinese', 'Contributor'], delimiter='\t')
df.drop(['Contributor'], axis=1, inplace=True)

In [ ]:
df.head()

,English,Chinese
0,Hi.,嗨。
1,Hi.,你好。
2,Run.,你用跑的。
3,Wait!,等等！
4,Hello!,你好。


## Tokenize

In [ ]:
encoder_tokenizer = AutoTokenizer.from_pretrained(encoder_model_name)
decoder_tokenizer = AutoTokenizer.from_pretrained(decoder_model_name)

In [ ]:
tok_df = df.copy()

encoder_tok_list = L(parallel_gen(TransformersTokenizer, df.Chinese, tokenizer=encoder_tokenizer)).sorted().itemgot(1)
tok_df.Chinese =  encoder_tok_list.map(lambda x: ' '.join(x)) # split tokens by ' '

decoder_tok_list = L(parallel_gen(TransformersTokenizer, df.English, tokenizer=decoder_tokenizer)).sorted().itemgot(1)
tok_df.English =  decoder_tok_list.map(lambda x: ' '.join(x)) # split tokens by ' '

tok_df.head()

,English,Chinese
0,Hi .,嗨 。
1,Hi .,你 好 。
2,Run .,你 用 跑 的 。
3,Wait !,等 等 ！
4,Hello !,你 好 。


In [ ]:
max(encoder_tok_list.map(len)), max(decoder_tok_list.map(len)) # max len of encoder, max len of decoder

(44, 36)

## Add is_valid

In [ ]:
is_valid = np.zeros(len(tok_df))
is_valid[:int(len(tok_df)*0.2)] = 1
np.random.RandomState(seed).shuffle(is_valid)
is_valid = is_valid.astype(np.bool)
tok_df['is_valid'] = is_valid

In [ ]:
tok_df

,English,Chinese,is_valid
0,Hi .,嗨 。,False
1,Hi .,你 好 。,False
2,Run .,你 用 跑 的 。,False
3,Wait !,等 等 ！,False
4,Hello !,你 好 。,False
...,...,...,...
21200,"Last Ġyear Ġin Ġthe ĠPhilippines , Ġearthquakes Ġand Ġtidal Ġwaves Ġresulted Ġin Ġthe Ġdeaths Ġof Ġmore Ġthan Ġ6 , 000 Ġpeople .",去 年 在 菲 律 宾 ， 地 震 和 海 啸 造 成 了 超 过 6000 人 的 死 亡 。,False
21201,"My Ġmother Ġspeaks ĠFrench Ġbetter Ġthan Ġmy Ġfather Ġspeaks ĠEnglish , Ġso Ġthey Ġusually Ġspeak Ġto Ġeach Ġother Ġin ĠFrench .",我 母 亲 的 法 语 比 我 父 亲 的 英 语 要 好 ， 所 以 他 们 通 常 用 法 语 交 流 。,False
21202,"Tom Ġdidn 't Ġknow Ġhow Ġto Ġtranslate Ġthe Ġword Ġ"" computer "" Ġbecause Ġthe Ġpeople Ġhe Ġwas Ġtalking Ġto Ġhad Ġnever Ġseen Ġone .",汤 姆 不 知 如 何 翻 译 [UNK] 计 算 机 [UNK] 一 词 ， 因 为 同 他 谈 话 的 人 从 未 见 过 一 台 。,False
21203,"Even Ġnow , ĠI Ġoccasionally Ġthink ĠI 'd Ġlike Ġto Ġsee Ġyou . ĠNot Ġthe Ġyou Ġthat Ġyou Ġare Ġtoday , Ġbut Ġthe Ġyou ĠI Ġremember Ġfrom Ġthe Ġpast .",即 使 是 现 在 ， 我 偶 尔 还 是 想 见 到 你 。 不 是 今 天 的 你 ， 而 是 我 记 忆 中 曾 经 的 你 。,True


## Save

In [ ]:
tok_df.to_csv(tok_data_loc, index=False)

In [ ]:
tok_df = pd.read_csv(tok_data_loc)

In [ ]:
tok_df

,English,Chinese,is_valid
0,Hi .,嗨 。,False
1,Hi .,你 好 。,False
2,Run .,你 用 跑 的 。,False
3,Wait !,等 等 ！,False
4,Hello !,你 好 。,False
...,...,...,...
21200,"Last Ġyear Ġin Ġthe ĠPhilippines , Ġearthquakes Ġand Ġtidal Ġwaves Ġresulted Ġin Ġthe Ġdeaths Ġof Ġmore Ġthan Ġ6 , 000 Ġpeople .",去 年 在 菲 律 宾 ， 地 震 和 海 啸 造 成 了 超 过 6000 人 的 死 亡 。,False
21201,"My Ġmother Ġspeaks ĠFrench Ġbetter Ġthan Ġmy Ġfather Ġspeaks ĠEnglish , Ġso Ġthey Ġusually Ġspeak Ġto Ġeach Ġother Ġin ĠFrench .",我 母 亲 的 法 语 比 我 父 亲 的 英 语 要 好 ， 所 以 他 们 通 常 用 法 语 交 流 。,False
21202,"Tom Ġdidn 't Ġknow Ġhow Ġto Ġtranslate Ġthe Ġword Ġ"" computer "" Ġbecause Ġthe Ġpeople Ġhe Ġwas Ġtalking Ġto Ġhad Ġnever Ġseen Ġone .",汤 姆 不 知 如 何 翻 译 [UNK] 计 算 机 [UNK] 一 词 ， 因 为 同 他 谈 话 的 人 从 未 见 过 一 台 。,False
21203,"Even Ġnow , ĠI Ġoccasionally Ġthink ĠI 'd Ġlike Ġto Ġsee Ġyou . ĠNot Ġthe Ġyou Ġthat Ġyou Ġare Ġtoday , Ġbut Ġthe Ġyou ĠI Ġremember Ġfrom Ġthe Ġpast .",即 使 是 现 在 ， 我 偶 尔 还 是 想 见 到 你 。 不 是 今 天 的 你 ， 而 是 我 记 忆 中 曾 经 的 你 。,True
